In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install lxml

In [5]:
import lxml

In [15]:
# gets CPI data
path = 'CPIAUCSL.csv'
df_CPI = pd.read_csv(path)

In [16]:
# gets base inflation value for end date of dataset
#BASE_YEAR = df_CPI[df_CPI['DATE'] == '2023-12-01']['CPIAUCSL']
# I'm not sure why above code isn't working, so I'm gonna do this manually
BASE_YEAR =  308.742

In [17]:
# creates a multiplier to be able to "iflation adjust" damages
df_CPI['CPI_Multiplier'] =  BASE_YEAR / df_CPI['CPIAUCSL'].astype(int)

In [18]:
df_CPI.head()

,DATE,CPIAUCSL,CPI_Multiplier
0,1947-01-01,21.48,14.702000
1,1947-02-01,21.62,14.702000
2,1947-03-01,22.00,14.033727
3,1947-04-01,22.00,14.033727
4,1947-05-01,21.95,14.702000


In [19]:
# gets CPI for housing specifically
path = 'CPIEHOUSE.csv'
df_housing = pd.read_csv(path)

In [20]:
# gets base year of 2023-12-01
# for same reason, I'll input manually
BASE_RATE = 343.991

In [21]:
# creates a multiplier to be able to "iflation adjust" damages
df_housing['Housing_Multiplier'] =  BASE_RATE / df_housing['CPIEHOUSE'].astype(int)

In [22]:
df_housing.head()

,DATE,CPIEHOUSE,Housing_Multiplier
0,1982-12-01,100.0,3.439910
1,1983-01-01,100.5,3.439910
2,1983-02-01,100.6,3.439910
3,1983-03-01,100.6,3.439910
4,1983-04-01,101.1,3.405851


In [23]:
inflation = df_housing.merge(df_CPI, on='DATE', how='left')

In [24]:
inflation.head()

,DATE,CPIEHOUSE,Housing_Multiplier,CPIAUCSL,CPI_Multiplier
0,1982-12-01,100.0,3.439910,97.7,3.182907
1,1983-01-01,100.5,3.439910,97.9,3.182907
2,1983-02-01,100.6,3.439910,98.0,3.150429
3,1983-03-01,100.6,3.439910,98.1,3.150429
4,1983-04-01,101.1,3.405851,98.8,3.150429


In [74]:
state = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area', match='State / territory')[0]
state

State / territory Total area          Land area           \
               State / territory      sq mi      km2     sq mi      km2   
0                         Alaska     665384  1723337    570641  1477953   
1                          Texas     268596   695662    261232   676587   
2                     California     163695   423967    155779   403466   
3                        Montana     147040   380831    145546   376962   
4                     New Mexico     121590   314917    121298   314161   
..                           ...        ...      ...       ...      ...   
56  Minor Outlying Islands[3][a]         16       41        16       41   
57                 Contiguous US    3120428  8081869   2954843  7653006   
58                     50 States    3796676  9833342   3531846  9147436   
59              50 States and DC    3796744  9833519   3531907  9147594   
60                 United States    3805927  9857306   3535932  9158022   

   Water area                 
        sq mi     km2      %  
0       94743  245383  14.2%  
1        7365   19075   2.7%  
2        7916   20501   4.8%  
3        1494    3869   1.0%  
4         292     757   0.2%  
..        ...     ...    ...  
56          0       0   0.0%  
57     165589  428865   5.3%  
58     264834  685907   7.0%  
59     264841  685926   7.0%  
60     269995  699284   7.1%  

[61 rows x 8 columns]

In [75]:
state_simple = pd.concat([state['State / territory'], state.xs('sq mi', axis=1, level=1)], axis=1)

In [80]:
population = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_historical_population', match='Name')[3]
population.head()

,Name,1960,1970,1980,1990,2000,2010,2020
0,Alabama,3266740,3444165,3893888,4040587,4447100,4779736,5024279
1,Alaska,226167,300382,401851,550043,626932,710231,733391
2,American Samoa,20051,27159,32297,46773,57291,55519,49710
3,Arizona,1302161,1770900,2718215,3665228,5130632,6392017,7151502
4,Arkansas,1786272,1923295,2286435,2350725,2673400,2915918,3011524


In [56]:
state_data = pd.merge(state_simple, population, left_on='State / territory', right_on='Name', how='inner')

In [57]:
state_data.head()

,State / territory,Total area,Land area,Water area,Name,1960,1970,1980,1990,2000,2010,2020
0,Alaska,665384,570641,94743,Alaska,226167,300382,401851,550043,626932,710231,733391
1,Texas,268596,261232,7365,Texas,9579677,11196730,14229191,16986510,20851820,25145561,29145505
2,California,163695,155779,7916,California,15717204,19953134,23667902,29760021,33871648,37253956,39538223
3,Montana,147040,145546,1494,Montana,674767,694409,786690,799065,902195,989415,1084225
4,New Mexico,121590,121298,292,New Mexico,951023,1016000,1302894,1515069,1819046,2059179,2117522


In [67]:
test = state_data.drop('Name', axis=1)
test = test.rename(columns={'State / territory': 'state',
                                  'Total area': 'total_area',
                                  'Land area': 'land_area',
                                  'Water area': 'water_area'})

In [69]:
test.head()

,state,total_area,land_area,water_area,1960,1970,1980,1990,2000,2010,2020
0,Alaska,665384,570641,94743,226167,300382,401851,550043,626932,710231,733391
1,Texas,268596,261232,7365,9579677,11196730,14229191,16986510,20851820,25145561,29145505
2,California,163695,155779,7916,15717204,19953134,23667902,29760021,33871648,37253956,39538223
3,Montana,147040,145546,1494,674767,694409,786690,799065,902195,989415,1084225
4,New Mexico,121590,121298,292,951023,1016000,1302894,1515069,1819046,2059179,2117522


In [85]:
path = 'https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations'
test = pd.read_html(path, match='Name')[0]
test.head()

Name Status of region                ISO  \
                       Name Status of region Unnamed: 2_level_1   
0  United States of America    Federal state         US USA 840   
1                   Alabama            State              US-AL   
2                    Alaska            State              US-AK   
3                   Arizona            State              US-AZ   
4                  Arkansas            State              US-AR   

                ANSI                                  USPS               USCG  \
  Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1   
0                 US                0.0                NaN                NaN   
1                 AL                1.0                 AL                 AL   
2                 AK                2.0                 AK                 AK   
3                 AZ                4.0                 AZ                 AZ   
4                 AR                5.0                 AR                 AR   

      GPO      AP Other abbreviations  
      GPO      AP Other abbreviations  
0    U.S.    U.S.              U.S.A.  
1    Ala.    Ala.                 NaN  
2  Alaska  Alaska              Ak.[2]  
3   Ariz.   Ariz.                 NaN  
4    Ark.    Ark.                 NaN

In [96]:
#test['Name']
#test.xs('Unnamed: 3_level_1', axis=1, level=1)
abv = pd.concat([test['Name'], test.xs('Unnamed: 3_level_1', axis=1, level=1)], axis=1)
abv.head()

,Name,ANSI
0,United States of America,US
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [97]:
pd.merge(state_simple, abv, left_on='State / territory',\
                          right_on='Name', how='inner')

,State / territory,Total area,Land area,Water area,Name,ANSI
0,Alaska,665384,570641,94743,Alaska,AK
1,Texas,268596,261232,7365,Texas,TX
2,California,163695,155779,7916,California,CA
3,Montana,147040,145546,1494,Montana,MT
4,New Mexico,121590,121298,292,New Mexico,NM
5,Arizona,113990,113594,396,Arizona,AZ
6,Nevada,110572,109781,791,Nevada,NV
7,Colorado,104094,103642,452,Colorado,CO
8,Oregon,98379,95988,2391,Oregon,OR
9,Wyoming,97813,97093,720,Wyoming,WY


I'm going to create functions that we can call

In [100]:
def create_state_data():
    # gets state area info
    state = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area',\
                         match='State / territory')[0]
    state_simple = pd.concat([state['State / territory'], state.xs('sq mi', axis=1, level=1)], axis=1)
    
    # gets state abbrevations
    names = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations',\
                       match='Name')[0]
    abv = pd.concat([names['Name'], names.xs('Unnamed: 3_level_1', axis=1, level=1)], axis=1)
    
    # combines state and abbrevations
    state_info = pd.merge(state_simple, abv, left_on='State / territory',\
                          right_on='Name', how='inner')
    state_info = state_info.drop('Name', axis=1).drop_duplicates()
    
    # gets state population info
    population = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_historical_population',\
                              match='Name')[3]
    
    # combines info
    state_data = pd.merge(state_info, population, left_on='State / territory',\
                          right_on='Name', how='inner')
    
    # cleans up columns
    df = state_data.drop('Name', axis=1)
    df = df.rename(columns={'State / territory': 'state',
                                  'Total area': 'total_area',
                                  'Land area': 'land_area',
                                  'Water area': 'water_area'})
    return df
    

In [99]:
create_state_data()

,state,total_area,land_area,water_area,ANSI,1960,1970,1980,1990,2000,2010,2020
0,Alaska,665384,570641,94743,AK,226167,300382,401851,550043,626932,710231,733391
1,Texas,268596,261232,7365,TX,9579677,11196730,14229191,16986510,20851820,25145561,29145505
2,California,163695,155779,7916,CA,15717204,19953134,23667902,29760021,33871648,37253956,39538223
3,Montana,147040,145546,1494,MT,674767,694409,786690,799065,902195,989415,1084225
4,New Mexico,121590,121298,292,NM,951023,1016000,1302894,1515069,1819046,2059179,2117522
5,Arizona,113990,113594,396,AZ,1302161,1770900,2718215,3665228,5130632,6392017,7151502
6,Nevada,110572,109781,791,NV,285278,488738,800493,1201833,1998257,2700551,3104614
7,Colorado,104094,103642,452,CO,1753947,2207259,2889964,3294394,4301261,5029196,5773714
8,Oregon,98379,95988,2391,OR,1768687,2091385,2633105,2842321,3421399,3831074,4237256
9,Wyoming,97813,97093,720,WY,330066,332416,469557,453588,493782,563626,576851
